<a href="https://colab.research.google.com/github/toandaominh1997/Mainspring_Hiring_Test/blob/master/Mainspring_Hiring_Test/Source%20Code/Mainspring_Hiring_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
flag_google_colab = True

In [45]:
if(flag_google_colab):
  from google.colab import drive
  drive.mount('/content/drive')
  !pip install lightgbm
  !pip install xgboost
  data_dir = "drive/My Drive/Dataset/Machine Learning Test(ZALORA)/"
else:
  data_dir = ""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Library Nescessary

In [0]:
import numpy as np
import pandas

# Preprocess Data

In [48]:
from scipy.sparse import hstack

def readTxt(filename):
  print(filename)
  lines = []
  with open(filename) as f:
    lines = f.readlines()
  
  return lines

train_normal = readTxt(data_dir+'data/training_data/normal_comments.txt')
stop_words = readTxt(data_dir+'data/training_data/normal_comments.txt')
train_sara = readTxt(data_dir+'data/training_data/sara_comments.txt')
test_normal= readTxt(data_dir+'data/test_data/nornal_comments.txt')
test_sara = readTxt(data_dir+'data/test_data/sara_comments.txt')

# Labels of Train Datasets
up_normal = np.ones((len(train_normal), 1))
up_sara = np.zeros((len(train_sara), 1))
y_train = np.concatenate((up_normal, up_sara), axis=0)

print(y_train.shape)
# Labels of Test Datasets
normal_test_labels = np.ones((len(test_normal), 1))
sara_test_labels = np.zeros((len(test_sara), 1))
y_test = np.concatenate((normal_test_labels, sara_test_labels), axis=0)

# X train of Datasets
train_normal.extend(train_sara)

print("Train normal1 = ", len(train_normal))
# X test of Datasets

test_normal.extend(test_sara)

all_text = train_normal
all_text.extend(test_normal)

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stop_words,
    ngram_range=(1, 1),
    norm='l2',
    min_df=0,
    smooth_idf=False,
    max_features=50000)
word_vectorizer.fit(all_text)
train_normal = readTxt(data_dir+'data/training_data/normal_comments.txt')
train_normal.extend(train_sara)

train_word_features = word_vectorizer.transform(train_normal)
test_word_features = word_vectorizer.transform(test_normal)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words=stop_words,
    ngram_range=(2, 6),
    norm='l2',
    min_df=0,
    smooth_idf=False,
    max_features=15000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_normal)

test_char_features = char_vectorizer.transform(test_normal)
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

print(train_features.shape)
print(test_features.shape)



drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/training_data/normal_comments.txt
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/training_data/normal_comments.txt
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/training_data/sara_comments.txt
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/test_data/nornal_comments.txt
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/test_data/sara_comments.txt
(75300, 1)
Train normal1 =  75300
drive/My Drive/Dataset/Machine Learning Test(ZALORA)/data/training_data/normal_comments.txt
(75300, 65000)
(9646, 65000)
(75300, 1)
(9646, 1)


In [25]:
X_train = train_features
X_test = test_features
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(75300, 65000)
(75300, 1)


# Model 1: logistic Regression

This is a classification problem with 2 classes. I try to use Logistic Regression for my model.</br>
As an optimization problem, binary class L2 penalized logistic regression minimizes the following cost function:<br>
$\min_{w, c} \frac{1}{2}w^T w + C \sum_{i=1}^n \log(\exp(- y_i (X_i^T w + c)) + 1) .$</br>
In my model, I choose C = 13.0 for best accuracy with model Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

def LogReg(X_train, y_train):
  log = LogisticRegression(C=13.0)
  log.fit(X_train, y_train)
  return log

t = time.time()
print("Fitting...")
model_logreg = LogReg(X_train, y_train)
print("Done!")
print("Time to train with LogisticRegression: ", time.time()-t)

Fitting...


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Done!
Time to train with LogisticRegression:  34.938544273376465


In [27]:
pred_train_logreg = model_logreg.predict(X_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, pred_train_logreg)))

pred_test_logreg = model_logreg.predict(X_test)
print('Testing accuracy is {}'.format(accuracy_score(y_test, pred_test_logreg)))


Training accuracy is 0.9576095617529881
Testing accuracy is 0.86377773170226


## Model 2: Lightgbm
In kaggle competitions, the model I use most is lightgbm, Xgboost. It use multiple decide tree to train model.
### What is Light GBM?
Light GBM is a gradient boosting framework that uses tree based learning algorithm.</br>

Light GBM grows tree vertically. It will choose the leaf with max delta loss to grow. When growing the same leaf, Leaf-wise algorithm can reduce more loss than a level-wise algorithm.


In [28]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

def LightGBM(X_train, y_train):
  model = LogisticRegression(solver='sag')
  sfm = SelectFromModel(model, threshold=0.2)

  _y_train = np.asarray(y_train)
  _y_train = _y_train.ravel()
  train_sparse_matrix = sfm.fit_transform(X_train, _y_train)
  train_sparse_matrix, valid_sparse_matrix, y_train_lgb, y_valid = train_test_split(train_sparse_matrix, _y_train, test_size=0.05, random_state=144)

  test_sparse_matrix = sfm.transform(X_test)
  d_train = lgb.Dataset(train_sparse_matrix, label=y_train_lgb)
  d_valid = lgb.Dataset(valid_sparse_matrix, label=y_valid)
  watchlist = [d_train, d_valid]
 
  params = {'learning_rate': 0.2,
              'application': 'binary',
              'num_leaves': 31,
              'verbosity': -1,
              'metric': 'auc',
              'data_random_seed': 2,
              'bagging_fraction': 0.8,
              'feature_fraction': 0.6,
              'nthread': 4,
              'lambda_l1': 1,
              'lambda_l2': 1}

  model = lgb.train(params, train_set=d_train, valid_sets=watchlist, verbose_eval=10)
  return model

model_lgb = LightGBM(X_train, y_train)



[10]	training's auc: 0.860414	valid_1's auc: 0.835863
[20]	training's auc: 0.884593	valid_1's auc: 0.856387
[30]	training's auc: 0.897923	valid_1's auc: 0.866279
[40]	training's auc: 0.90838	valid_1's auc: 0.872598
[50]	training's auc: 0.916362	valid_1's auc: 0.875681
[60]	training's auc: 0.923006	valid_1's auc: 0.877774
[70]	training's auc: 0.928684	valid_1's auc: 0.87964
[80]	training's auc: 0.933898	valid_1's auc: 0.879411
[90]	training's auc: 0.938855	valid_1's auc: 0.879285
[100]	training's auc: 0.942996	valid_1's auc: 0.878975


In [29]:
pred_train_lgb = model_logreg.predict(X_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, pred_train_lgb)))

pred_test_lgb = model_logreg.predict(X_test)
print('Testing accuracy is {}'.format(accuracy_score(y_test, pred_test_lgb)))

Training accuracy is 0.9576095617529881
Testing accuracy is 0.86377773170226


## Model 3: XGBoost

In [0]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

x_train, X_valid, y_train_xgb, y_valid = train_test_split(
        X_train, y_train, test_size=0.25, random_state=23)
def XGBoost(X_train, X_valid, y_train_xgb, y_valid):
  xgb_params = {'eta': 0.3, 
              'max_depth': 5, 
              'subsample': 0.8, 
              'colsample_bytree': 0.8, 
              'objective': 'binary:logistic', 
              'eval_metric': 'auc', 
              'seed': 23
             }
  d_train = xgb.DMatrix(X_train, y_train_xgb)
  d_valid = xgb.DMatrix(X_valid, y_valid)
  watchlist = [(d_valid, 'valid')]
  model = xgb.train(xgb_params, d_train, 200, watchlist, verbose_eval=False, early_stopping_rounds=30)
  return model

model_xgb = XGBoost(x_train, X_valid, y_train_xgb, y_valid)


In [33]:
d_train = xgb.DMatrix(X_train)
pred_train_xgb = model_xgb.predict(d_train)
print('Training accuracy is {}'.format(accuracy_score(y_train, pred_train_xgb.round())))

d_test = xgb.DMatrix(X_test)
pred_test_xgb = model_xgb.predict(d_test)
print('Testing accuracy is {}'.format(accuracy_score(y_test, pred_test_xgb.round())))

Training accuracy is 0.9137317397078353
Testing accuracy is 0.8584905660377359


In [43]:
from sklearn.metrics import precision_recall_fscore_support


print(precision_recall_fscore_support(pred_test_logreg, y_test, average='weighted'))
print(precision_recall_fscore_support(pred_test_lgb.round(), y_test, average='weighted'))
print(precision_recall_fscore_support(pred_test_xgb.round(), y_test, average='weighted'))


(0.8955840979633506, 0.86377773170226, 0.8720931814589512, None)
(0.8955840979633506, 0.86377773170226, 0.8720931814589512, None)
(0.9091837012778481, 0.8584905660377359, 0.8710123323397941, None)


In [39]:
from sklearn.metrics import precision_recall_fscore_support
y_true = np.array(['cat', 'dog', 'pig', 'cat', 'dog', 'pig'])
y_pred = np.array(['cat', 'pig', 'dog', 'cat', 'cat', 'dog'])
precision_recall_fscore_support(y_true, y_pred, average='macro')

(0.2222222222222222, 0.3333333333333333, 0.26666666666666666, None)